<a href="https://colab.research.google.com/github/dprasadk/Bronchitis-Detection-for-Apollo/blob/master/Bronchitis_Detection_for_Apollo_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle

In [0]:
from googleapiclient.discovery import build
import io, os, glob
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

In [0]:
auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
!kaggle competitions download -c bronchitis -p /content/kaggle

Train_Labels.csv: Skipping, found more recently modified local copy (use --force to force download)
Sample_Submission.csv: Skipping, found more recently modified local copy (use --force to force download)
Test_Images.zip: Skipping, found more recently modified local copy (use --force to force download)
Train_Images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
os.chdir('/content/kaggle')
!unzip  Train_Images.zip -d /content/kaggle/Train
!unzip  Test_Images.zip -d /content/kaggle/Validation

In [0]:
ls /content/kaggle

fine_tuned_model.h5    Sample_Submission.csv  Train/            Validation/
fine_tuned_model.json  Test_Images.zip        Train_Images.zip
fine_tuned_weights.h5  top_model.h5           Train_Labels.csv


In [0]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from keras.regularizers import l1_l2
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dropout, GlobalAveragePooling2D, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
import PIL
import math
import pandas as pd

In [0]:
img_width, img_height = (299, 299)
weight_path = '/content/kaggle/fine_tuned_weights.h5'
top_model_weight_path = '/content/kaggle/top_model.h5'
train_dir = '/content/kaggle/Train'
validation_dir = '/content/kaggle/Validation'
# epochs = 150
# batch_size = 128
epochs = 15
batch_size = 32

nb_train_samples =640
nb_validation_samples = 160

In [0]:
base_model =InceptionV3(weights= 'imagenet', include_top= False, input_shape=(299,299,3))
print ("Model Loaded.")

Model Loaded.


In [0]:
top_model= Sequential()
top_model.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(4, input_shape =(2,), activation='relu'))
top_model.add(Dense(4,  activation='relu'))
top_model.add(Dense(2, activation='softmax')) 

In [0]:
top_model.save_weights(top_model_weight_path)
top_model.load_weights(top_model_weight_path)

In [0]:
model= Model(inputs= base_model.input, outputs= top_model(base_model.output))

In [0]:
for layer in model.layers[:172]:
    layer.trainable=False
for layer in model.layers[172:]:
    layer.trainable=True

In [0]:
model.compile(optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
train_datagen= ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True)

In [0]:
test_datagen= ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True)

In [0]:
train_generator= train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 640 images belonging to 1 classes.


In [0]:
validation_generator= test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 160 images belonging to 1 classes.


In [0]:
callbacks_list=[ModelCheckpoint(weight_path, monitor='val_acc',verbose=1,save_best_only=True),
            EarlyStopping(monitor='val_acc',patience=10,verbose=0)]

In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch = int(math.ceil(nb_train_samples / batch_size)),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps = int(math.ceil(nb_validation_samples / batch_size)), callbacks=callbacks_list)

Epoch 1/15
20/20 [==============================] - 150s 7s/step - loss: 0.3580 - acc: 1.0000 - val_loss: 0.3280 - val_acc: 1.0000

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to /content/kaggle/fine_tuned_weights.h5
Epoch 2/15
20/20 [==============================] - 87s 4s/step - loss: 0.1996 - acc: 1.0000 - val_loss: 0.1860 - val_acc: 1.0000

Epoch 00002: val_acc did not improve from 1.00000
Epoch 3/15
20/20 [==============================] - 117s 6s/step - loss: 0.1174 - acc: 1.0000 - val_loss: 0.1371 - val_acc: 1.0000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/15
20/20 [==============================] - 117s 6s/step - loss: 0.0805 - acc: 1.0000 - val_loss: 0.1022 - val_acc: 1.0000

Epoch 00004: val_acc did not improve from 1.00000
Epoch 5/15
20/20 [==============================] - 117s 6s/step - loss: 0.0601 - acc: 1.0000 - val_loss: 0.0831 - val_acc: 1.0000

Epoch 00005: val_acc did not improve from 1.00000
Epoch 6/15
20/20 [=================

20/20 [==============================] - 117s 6s/step - loss: 0.0399 - acc: 1.0000 - val_loss: 0.0627 - val_acc: 1.0000

Epoch 00007: val_acc did not improve from 1.00000
Epoch 8/15
20/20 [==============================] - 117s 6s/step - loss: 0.0334 - acc: 1.0000 - val_loss: 0.0523 - val_acc: 1.0000

Epoch 00008: val_acc did not improve from 1.00000
Epoch 9/15
20/20 [==============================] - 116s 6s/step - loss: 0.0290 - acc: 1.0000 - val_loss: 0.0485 - val_acc: 1.0000

Epoch 00009: val_acc did not improve from 1.00000
Epoch 10/15
20/20 [==============================] - 118s 6s/step - loss: 0.0257 - acc: 1.0000 - val_loss: 0.0433 - val_acc: 1.0000

Epoch 00010: val_acc did not improve from 1.00000
Epoch 11/15
20/20 [==============================] - 118s 6s/step - loss: 0.0227 - acc: 1.0000 - val_loss: 0.0396 - val_acc: 1.0000

Epoch 00011: val_acc did not improve from 1.00000


In [0]:
model_json = model.to_json()
 
with open("fine_tuned_model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("fine_tuned_model.h5")

In [0]:
ls /content/kaggle

fine_tuned_model.h5    Sample_Submission.csv  Train/            Validation/
fine_tuned_model.json  Test_Images.zip        Train_Images.zip
fine_tuned_weights.h5  top_model.h5           Train_Labels.csv


In [0]:
json_file = open("fine_tuned_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
 
loaded_model.load_weights("fine_tuned_model.h5")
 

In [0]:
loaded_model.compile(optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])


ev_score = loaded_model.evaluate_generator(validation_generator, nb_validation_samples/batch_size)

pd_scores = loaded_model.predict_generator(validation_generator, nb_validation_samples/batch_size, 
                                max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [0]:
#@title Evaluation Score
 ev_score

[0.037056367844343185, 1.0]

In [0]:
#@title Predicted Score for 160 prediction row is 0.5
pd_scores

array([[0.02012423, 0.9798758 ],
       [0.02880235, 0.97119766],
       [0.02708004, 0.97291994],
       [0.02309873, 0.97690123],
       [0.04457535, 0.9554246 ],
       [0.05664415, 0.94335586],
       [0.03872984, 0.9612702 ],
       [0.04482285, 0.9551771 ],
       [0.04402927, 0.95597076],
       [0.08658952, 0.9134104 ],
       [0.01709818, 0.9829018 ],
       [0.01221646, 0.9877836 ],
       [0.03717671, 0.96282333],
       [0.05221851, 0.94778144],
       [0.01753783, 0.98246217],
       [0.04109515, 0.95890486],
       [0.02327865, 0.9767213 ],
       [0.0254886 , 0.97451144],
       [0.06808145, 0.93191856],
       [0.05299407, 0.9470059 ],
       [0.01013971, 0.98986036],
       [0.06378959, 0.9362104 ],
       [0.07646646, 0.9235335 ],
       [0.04940748, 0.9505925 ],
       [0.02170874, 0.9782913 ],
       [0.02326675, 0.97673327],
       [0.02929698, 0.97070307],
       [0.02361066, 0.97638935],
       [0.01614433, 0.98385566],
       [0.02321398, 0.976786  ],
       [0.

In [0]:
#@title Actual predicted Score
  pd_scores.mean()

0.5

In [0]:
#@title Build Test_Labels.csv file
getEachImgPrediction= []

for i in range(len(pd_scores)):
  if pd_scores[i][0] > 0.03:
    getEachImgPrediction.append(1)
  else:
     getEachImgPrediction.append(0)

In [0]:
directory  = "/content/kaggle/Validation/Test_Images"

for r, dirs, pngTestfiles in os.walk(directory):
  print(pngTestfiles)

['CR_0326.png', 'MR_0348.png', 'CR_0148.png', 'CR_0457.png', 'CR_0662.png', 'CR_0436.png', 'CR_0266.png', 'CR_0304.png', 'CR_0111.png', 'CR_0022.png', 'CR_0083.png', 'CR_0003.png', 'CR_0464.png', 'CR_0100.png', 'CR_0020.png', 'CR_0447.png', 'CR_0411.png', 'CR_0353.png', 'MR_0013.png', 'MR_0040.png', 'CR_0146.png', 'MR_0228.png', 'CR_0397.png', 'CR_0101.png', 'CR_0504.png', 'CR_0421.png', 'CR_0392.png', 'MR_0070.png', 'CR_0391.png', 'CR_0283.png', 'MR_0126.png', 'Test_Labels.csv.gz', 'MR_0052.png', 'CR_0615.png', 'CR_0489.png', 'CR_0365.png', 'CR_0219.png', 'MR_0027.png', 'CR_0157.png', 'CR_0035.png', 'CR_0492.png', 'CR_0395.png', 'MR_0203.png', 'CR_0348.png', 'CR_0325.png', 'CR_0240.png', 'CR_0214.png', 'CR_0024.png', 'CR_0649.png', 'CR_0300.png', 'CR_0637.png', 'CR_0173.png', 'CR_0096.png', 'CR_0536.png', 'CR_0514.png', 'CR_0600.png', 'MR_0170.png', 'CR_0547.png', 'CR_0143.png', 'MR_0275.png', 'CR_0497.png', 'CR_0315.png', 'CR_0216.png', 'CR_0568.png', 'CR_0396.png', 'MR_0023.png', 'C

In [0]:
os.chdir(directory)
splitPNG = []
for pngTestfiles in glob.glob("*.png"):
  splitPNG.append( os.path.splitext(pngTestfiles)[0] )
  print(splitPNG)

['CR_0326']
['CR_0326', 'MR_0348']
['CR_0326', 'MR_0348', 'CR_0148']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662', 'CR_0436']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662', 'CR_0436', 'CR_0266']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662', 'CR_0436', 'CR_0266', 'CR_0304']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662', 'CR_0436', 'CR_0266', 'CR_0304', 'CR_0111']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662', 'CR_0436', 'CR_0266', 'CR_0304', 'CR_0111', 'CR_0022']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662', 'CR_0436', 'CR_0266', 'CR_0304', 'CR_0111', 'CR_0022', 'CR_0083']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662', 'CR_0436', 'CR_0266', 'CR_0304', 'CR_0111', 'CR_0022', 'CR_0083', 'CR_0003']
['CR_0326', 'MR_0348', 'CR_0148', 'CR_0457', 'CR_0662', 'CR_0436', 'CR_0266', 'CR_0304', 'CR_0111', 'CR_0022', 'CR_0083', 'CR_0003

In [0]:
buildTest_ImageCSV = pd.DataFrame(
    {'Images': splitPNG,
     'Labels': getEachImgPrediction 
    })


In [0]:
buildTest_ImageCSV

,Images,Labels
0,CR_0326,0
1,MR_0348,0
2,CR_0148,0
3,CR_0457,0
4,CR_0662,1
5,CR_0436,1
6,CR_0266,1
7,CR_0304,1
8,CR_0111,1
9,CR_0022,1


In [0]:
buildTest_ImageCSV.to_csv('Test_Labels.csv', index = False)

In [0]:
ls /content/kaggle

fine_tuned_model.h5    Sample_Submission.csv  Train/            Validation/
fine_tuned_model.json  Test_Images.zip        Train_Images.zip  ziKdSdWX
fine_tuned_weights.h5  top_model.h5           Train_Labels.csv


In [0]:
!rm "Test_Labels.csv.zip"

In [0]:
!find /content/kaggle -type f \( -name "*.class" -o -name '*.csv.gz' \) -delete

In [0]:
!gzip Test_Labels.csv

In [0]:
from google.colab import files
files.download('Test_Labels.csv.gz')

In [0]:
#@title Kaggle Auto Submission 
!kaggle competitions submit -c bronchitis -f Test_Labels.csv.gz -m "Submission Details"

Successfully submitted to Bronchitis Detection for Apollo

In [0]:
!kaggle competitions submissions -c bronchitis

fileName                                          date                 description                 status    publicScore  privateScore  
------------------------------------------------  -------------------  --------------------------  --------  -----------  ------------  
Test_Labels.csv.gz                                2018-05-09 06:20:26  Submission Details          complete  0.50625      None          
Test_Labels.csv.gz                                2018-05-09 06:18:38  Submission Details          complete  0.50625      None          
Test_Labels.csv.gz                                2018-05-09 06:16:50  Submission Details          complete  0.48125      None          
Test_Labels.csv.gz                                2018-05-09 06:05:12  Submission Details          error     None         None          
Test_Labels.csv                                   2018-05-09 03:28:27  None                        complete  0.45625      None          
Test_Labels.rar                          